# Colab 환경 설정

In [9]:
!pip install -q -U "tensorflow-text==2.8.*"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 37.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 498.0/498.0 MB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 54.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 96.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 462.3/462.3 KB 38.7 MB/s eta 0:00:00


In [ ]:
!pip install -q tf-models-official==2.7.0

In [7]:
!pip install transformers
!pip install pad_sequences

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [11]:
import tensorflow as tf
import torch

from transformers import BertTokenizer
from transformers import BertForSequenceClassification, AdamW, BertConfig
from transformers import get_linear_schedule_with_warmup
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras_preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np
import random
import time
import datetime

In [12]:
n_devices = torch.cuda.device_count()
print(n_devices)

for i in range(n_devices):
    print(torch.cuda.get_device_name(i))

1
Tesla T4


# 데이터셋 불러오기

## 학습 데이터 불러오기

In [13]:
df = pd.read_csv('/content/drive/MyDrive/동아리_스터디_대외활동/Solux/2학기/Solux_Honey_News/crawling/result_crawling/preprocessed_data.csv')
df

,title,news,new_article
0,윤 대통령 “북 무인기 한 대 왔으면 우린 2~3대 보내라”,중앙일보,윤석열 대통령 28일 북한 도발 에도 확실하게 응징 보복 라 그것 가장 강력한 수단...
1,"노웅래 체포안 부결...與 ""이재명 방탄 예고편""",중앙일보,저번 주셨는데 뭘 주냐 저번 그거 잘 쓰고 있는데 라고 말 하는 노웅래 민주당 의원...
2,이재명 때린 17년전 이재명? '지자체 부정부패' 논문 재조명,중앙일보,이재명 민주당 대표 성남 FC 후 원금 의혹 관련 해 검찰 수사 받는 가운데 17년...
3,"[단독] TBS 직원 10명중 6명 ""김어준 방송, 중립적이지 않다""",중앙일보,정치 적 편향 논란 끊이지 않던 TBS 교통 방송 라디오 프로그램 김어준 뉴스 공장...
4,미사일 안쏘고 韓 흔든 北…대박난 '회색지대 도발' 더 세진다,중앙일보,북한 대남 도발 날로 교묘해지고 핵 ㆍ 미사일 같은 고 강도 도발 에다 지난 26일...
...,...,...,...
395,43%가 ‘사표’…“소선거구제는 썩은 그릇에 국물 조금 붓는 것”,한겨레,2020년 치러진 21 대 총선 참여 한 유권자 2874만 1408 표 가운데 10...
396,윤 대통령 지지율 41.2%…2주 연속 상승 [리얼미터],한겨레,윤석열 대통령 국정 지지율 2 주 연속 상승 해 41 대를 기록 했다 리얼미터 미디...
397,"민주, ‘이재명 수사’ 검사 16명 공개…국힘 “헌법 질서에 도전”",한겨레,민주당 이재명 대표 관련 한 수사 진행 중인 검사 사진 이름 담긴 자료 만들어 전국...
398,노란봉투법·양곡관리법·차별금지법…여야 이견 속 해 넘길판,한겨레,여야 지난 24일 638조 7천억원 규모 내년 도 예산안 부수 법안 지각 처리 했지...


보수 정당: 1 조선일보, 중앙일보

진보 정당: 0 한겨례, 경향

In [14]:
df.loc[(df['news'] == '조선일보') | (df['news'] == '중앙일보'), 'news'] = 1

In [15]:
df.loc[(df['news'] == '한겨레') | (df['news'] == '경향'), 'news'] = 0
df

,title,news,new_article
0,윤 대통령 “북 무인기 한 대 왔으면 우린 2~3대 보내라”,1,윤석열 대통령 28일 북한 도발 에도 확실하게 응징 보복 라 그것 가장 강력한 수단...
1,"노웅래 체포안 부결...與 ""이재명 방탄 예고편""",1,저번 주셨는데 뭘 주냐 저번 그거 잘 쓰고 있는데 라고 말 하는 노웅래 민주당 의원...
2,이재명 때린 17년전 이재명? '지자체 부정부패' 논문 재조명,1,이재명 민주당 대표 성남 FC 후 원금 의혹 관련 해 검찰 수사 받는 가운데 17년...
3,"[단독] TBS 직원 10명중 6명 ""김어준 방송, 중립적이지 않다""",1,정치 적 편향 논란 끊이지 않던 TBS 교통 방송 라디오 프로그램 김어준 뉴스 공장...
4,미사일 안쏘고 韓 흔든 北…대박난 '회색지대 도발' 더 세진다,1,북한 대남 도발 날로 교묘해지고 핵 ㆍ 미사일 같은 고 강도 도발 에다 지난 26일...
...,...,...,...
395,43%가 ‘사표’…“소선거구제는 썩은 그릇에 국물 조금 붓는 것”,0,2020년 치러진 21 대 총선 참여 한 유권자 2874만 1408 표 가운데 10...
396,윤 대통령 지지율 41.2%…2주 연속 상승 [리얼미터],0,윤석열 대통령 국정 지지율 2 주 연속 상승 해 41 대를 기록 했다 리얼미터 미디...
397,"민주, ‘이재명 수사’ 검사 16명 공개…국힘 “헌법 질서에 도전”",0,민주당 이재명 대표 관련 한 수사 진행 중인 검사 사진 이름 담긴 자료 만들어 전국...
398,노란봉투법·양곡관리법·차별금지법…여야 이견 속 해 넘길판,0,여야 지난 24일 638조 7천억원 규모 내년 도 예산안 부수 법안 지각 처리 했지...


In [16]:
df.news.unique()

array([1, 0], dtype=object)

In [22]:
df['news'] = df['news'].astype(int)
df['news']

0      1
1      1
2      1
3      1
4      1
      ..
395    0
396    0
397    0
398    0
399    0
Name: news, Length: 400, dtype: int64

## Train set / Test set으로 나누기

In [23]:
# train dataset
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df[['title', 'new_article']], df['news'])
# X_train, X_valid, y_train, y_valid = train_test_split(X_train_full, y_train_full)

train = pd.concat([X_train,y_train],1)
test = pd.concat([X_test,y_test],1)

<ipython-input-23-505e853eba5d>:7: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  train = pd.concat([X_train,y_train],1)
<ipython-input-23-505e853eba5d>:8: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  test = pd.concat([X_test,y_test],1)


# Train set 전처리

## 각 문장마다 [CLS]와 [SEP]를 붙여주기

In [24]:
sentences = ["[CLS] " + str(s) + " [SEP]" for s in train.new_article]
sentences[:5]

['[CLS] 이재명 민주당 대표 29일 김근태 전 열린 우리당 의장 11 주기 추모 미사 참석 해 수십 간 의장 님 같은 분 희생 고통 통해 서 어렵게 쌓아 올린 민주주의 사방 무너지고 고 말 했다 대표 는 날 오전 11시 서울 도봉구 창동 성당 열린 추모 미사 나라 근간 인 민생 경제 백 척 간 두 위기 이고 한반도 다시 공포 그림자 짙게 드리워지고 위기 아닌 곳 찾기 힘들다 면서 이같이 말 했다 대표 는 공정 원칙 상실한 권력 폭주 강력하게 맞서겠다 면서 윤석열 정부 겨냥 했다 는 의장 님 께서는 누군가 해야 김근태 하겠다 말씀 자주 주셨다 면서 이제 김근태 되겠다 누군가 해야 이재명 민주당 앞장서겠다 고 밝혔다 앞서 대표 는 날 국회 열린 검찰 인권 침해 수사 문제점 제도 적 대책 마련 위 한 토론회 에서도 윤석열 정권 검찰 공익 대변자 라는 책무 망각 하고 민주주의 파괴 하는 도구로 전락 했다 고 주장 했다 대표 는 공 정성 완전히 상실한 정권 폭주 결연히 맞서야 한다 면서 일부 검찰 행태 매우 불공정 하고 편향 되고 조작 서슴지 않는 범죄 적 행위 행 하고 고 말 했다 대표 는 토론회 직후 1월 첫째 주 검찰 출석 하는 게 맞느냐 는 기자 질문 에는 대통령 가족 대한 수사 는 하는지도 관심 가져주기 바란다 고 응수 했다 대표 는 새해 첫날 경남 김해 봉하마을 찾아 노무현 전 대통령 묘역 참배 하고 날인 2일 경남 양산 평산 마을 찾아 문재인 전 대통령 방한 다 문재인 정부 인사 검찰 수사 선상 오른 가운데 대표 문 전 대통령 검찰 규탄 하는 메시지 낼 지 관심 모인다 박찬 대 민주당 의원 은 날 MBC 라디오 김종배 시선집중 문 전 대통령 이나 대표 도 검찰 독재 공화국 내 지금 엄청난 탄압 받고 있는 상황 이라면서 현 정세 대한 의견 은 서로 나눌 수 는 있지 않을까 생각 한다 고 말 했다 검찰 소환 통보 받은 대표 문 전 대통령 만나 정부 비판 하면 지 지층 결집 효과 있을 이라는 분석 나온다 대표 는 날 새해 맞아 국민 당원 보낸 연하 장 야당

## 서브워드 토크나이저: WordPiece

In [25]:
# 샘플 토크나이징
import pandas as pd
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained("bert-base-multilingual-cased", do_lower_case=False)
result = tokenizer.tokenize('안녕하세요')
print(result)

Downloading:   0%|          | 0.00/996k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/625 [00:00<?, ?B/s]

['안', '##녕', '##하', '##세', '##요']


In [26]:
# 전체 데이터에 토크나이징 수행
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased', do_lower_case=False)
tokenized_texts = [tokenizer.tokenize(s) for s in sentences]

In [27]:
# 토크나이징 잘 되었는지 확인하기
print(sentences[0])  #토크나이징 전
print(tokenized_texts[0]) #토크나이징 후

[CLS] 이재명 민주당 대표 29일 김근태 전 열린 우리당 의장 11 주기 추모 미사 참석 해 수십 간 의장 님 같은 분 희생 고통 통해 서 어렵게 쌓아 올린 민주주의 사방 무너지고 고 말 했다 대표 는 날 오전 11시 서울 도봉구 창동 성당 열린 추모 미사 나라 근간 인 민생 경제 백 척 간 두 위기 이고 한반도 다시 공포 그림자 짙게 드리워지고 위기 아닌 곳 찾기 힘들다 면서 이같이 말 했다 대표 는 공정 원칙 상실한 권력 폭주 강력하게 맞서겠다 면서 윤석열 정부 겨냥 했다 는 의장 님 께서는 누군가 해야 김근태 하겠다 말씀 자주 주셨다 면서 이제 김근태 되겠다 누군가 해야 이재명 민주당 앞장서겠다 고 밝혔다 앞서 대표 는 날 국회 열린 검찰 인권 침해 수사 문제점 제도 적 대책 마련 위 한 토론회 에서도 윤석열 정권 검찰 공익 대변자 라는 책무 망각 하고 민주주의 파괴 하는 도구로 전락 했다 고 주장 했다 대표 는 공 정성 완전히 상실한 정권 폭주 결연히 맞서야 한다 면서 일부 검찰 행태 매우 불공정 하고 편향 되고 조작 서슴지 않는 범죄 적 행위 행 하고 고 말 했다 대표 는 토론회 직후 1월 첫째 주 검찰 출석 하는 게 맞느냐 는 기자 질문 에는 대통령 가족 대한 수사 는 하는지도 관심 가져주기 바란다 고 응수 했다 대표 는 새해 첫날 경남 김해 봉하마을 찾아 노무현 전 대통령 묘역 참배 하고 날인 2일 경남 양산 평산 마을 찾아 문재인 전 대통령 방한 다 문재인 정부 인사 검찰 수사 선상 오른 가운데 대표 문 전 대통령 검찰 규탄 하는 메시지 낼 지 관심 모인다 박찬 대 민주당 의원 은 날 MBC 라디오 김종배 시선집중 문 전 대통령 이나 대표 도 검찰 독재 공화국 내 지금 엄청난 탄압 받고 있는 상황 이라면서 현 정세 대한 의견 은 서로 나눌 수 는 있지 않을까 생각 한다 고 말 했다 검찰 소환 통보 받은 대표 문 전 대통령 만나 정부 비판 하면 지 지층 결집 효과 있을 이라는 분석 나온다 대표 는 날 새해 맞아 국민 당원 보낸 연하 장 야당 파

In [28]:
# 문장의 최대 시퀀스를 설정하여 정수 인코딩과 제로 패딩을 수행해준다.
MAX_LEN = 128 #최대 시퀀스 길이 설정
# 정수 인코딩
input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
# 제로 패딩
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

In [29]:
input_ids[7]

array([   101,   6631,   9986,  80331,  11102,   9069,  11287,   9130,
        11513,   7908,   3476,  70672,   9323,  48036,  58466,   8933,
        13890,   9758,  51431,   9954,  89045,  78123,   9638,  37819,
         9519, 119285,   9732,   9318,  61250,   9435,  23466,   8872,
         9287,  16985,  72087,   9901,  12092,   9559,  12092,   8885,
        82512,   9574, 118782,  17360,   9095,  14646,   9960,  34951,
        18471,   9706,  66815,   9626,  17360,  10462,   9405,  24989,
         9657,  73380,   9034, 119007,   9632,  10017,  27023,  67527,
        10739,  39218, 103611,    122,   9565,  21711,   9043,   9583,
        14523,  67313,   8985,  12030,  56047,   9366,  11102,   9095,
        14523,   9414,   9059,  41521,  12692,   9847,  12092,  22458,
        12945,  18392,  23032,  11274,  11517,   9323,  12945,   9954,
         9083,  33378,   9379,  33323,  23622,   9565,  21928,   9632,
         9059,  20309,   9954,   9069, 119187,   9949,  48549,  17138,
      

## 어텐션 마스크
- 0 값을 가지는 패딩 토큰에 대해서 어텐션 연산을 불필요하게 수행하지 않도록 단어와 패딩 토큰을 구분할 수 있게 알려주는 것

In [30]:
attention_masks = []

for seq in input_ids:
    seq_mask = [float(i>0) for i in seq]
    attention_masks.append(seq_mask)

In [31]:
print(attention_masks[7])

[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]


## Train set를 훈련셋과 검증셋으로 분리하기
어텐션 마스크도 분리!

In [32]:
labels = train['news'].values
# labels = labels.astype(np.int)

train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(input_ids, labels, random_state=2, test_size=0.1)
train_masks, validation_masks, _, _ = train_test_split(attention_masks, input_ids, random_state=2000, test_size=0.1)

train_labels.astype('float64')

train_inputs = torch.tensor(train_inputs)
train_labels = torch.tensor(train_labels)
train_masks = torch.tensor(train_masks)
validation_inputs = torch.tensor(validation_inputs)
validation_labels = torch.tensor(validation_labels)
validation_masks = torch.tensor(validation_masks)

In [33]:
# 배치 사이즈 32로 설정하고, 입력데이터, 어텐션 마스크, 라벨을 하나의 데이터로 묶어 train_dataloader, validation_dataloader라는 입력 데이터를 생성해준다.
batch_size = 32

train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)

# Test set 전처리

In [34]:
sentences = ["[CLS] " + str(s) + " [SEP]" for s in test.new_article]

In [35]:
# [CLS] + 문장 + [SEP]
sentences = ["[CLS] " + str(sentence) + " [SEP]" for sentence in sentences]

# 라벨 데이터
labels = test['news'].values

# Word 토크나이저 토큰화
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased', do_lower_case=False)
tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]

# 시퀀스 설정 및 정수 인덱스 변환 & 패딩
MAX_LEN = 128
input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

# 어텐션 마스크
attention_masks = []
for seq in input_ids:
    seq_mask = [float(i>0) for i in seq]
    attention_masks.append(seq_mask)
    
# 파이토치 텐서로 변환
test_inputs = torch.tensor(input_ids)
labels = labels.astype(np.int)
test_labels = torch.tensor(labels)
test_masks = torch.tensor(attention_masks)

# 배치 사이즈 설정 및 데이터 설정
batch_size = 32
test_data = TensorDataset(test_inputs, test_masks, test_labels)
test_sampler = RandomSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=batch_size)

<ipython-input-35-91965f42e0e7>:24: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  labels = labels.astype(np.int)


In [36]:
len(test_inputs), len(test_masks), len(test_labels)

(100, 100, 100)

# BERT 모델 불러오기

In [37]:
# GPU 설정
if torch.cuda.is_available():    
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print('No GPU available, using the CPU instead.')

There are 1 GPU(s) available.
We will use the GPU: Tesla T4


In [38]:
# pretrain된 BERT 모델을 불러오자
model = BertForSequenceClassification.from_pretrained("bert-base-multilingual-cased", num_labels=2)
model.cuda()

Downloading:   0%|          | 0.00/714M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model ch

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elemen

In [39]:
# 하이퍼 파라미터 설정해주기

# 옵티마이저
optimizer = AdamW(model.parameters(),
                  lr = 2e-5, # 학습률(learning rate)
                  eps = 1e-8 
                )

# 에폭수
epochs = 4

# 총 훈련 스텝 : 배치반복 횟수 * 에폭
total_steps = len(train_dataloader) * epochs

# 스케줄러 생성
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0,
                                            num_training_steps = total_steps)

/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


# 모델 학습

In [160]:
# 정확도 계산 함수
def flat_accuracy(preds, labels):
    
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    return np.sum(pred_flat == labels_flat) / len(labels_flat)
    
    
# 시간 표시 함수
def format_time(elapsed):

    # 반올림
    elapsed_rounded = int(round((elapsed)))
    
    # hh:mm:ss으로 형태 변경
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [161]:
#랜덤시드 고정
seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

#그래디언트 초기화
model.zero_grad()

# 학습
for epoch_i in range(0, epochs):
    
    # ========================================
    #               Training
    # ========================================
    
    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # 시작 시간 설정
    t0 = time.time()

    # 로스 초기화
    total_loss = 0

    # 훈련모드로 변경
    model.train()
        
    # 데이터로더에서 배치만큼 반복하여 가져옴
    for step, batch in enumerate(train_dataloader):
        # 경과 정보 표시
        if step % 500 == 0 and not step == 0:
            elapsed = format_time(time.time() - t0)
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        # 배치를 GPU에 넣음
        batch = tuple(t.to(device) for t in batch)
        
        # 배치에서 데이터 추출
        b_input_ids, b_input_mask, b_labels = batch

        # Forward 수행                
        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask, 
                        labels=b_labels)
        
        # 로스 구함
        loss = outputs[0]

        # 총 로스 계산
        total_loss += loss.item()

        # Backward 수행으로 그래디언트 계산
        loss.backward()

        # 그래디언트 클리핑
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # 그래디언트를 통해 가중치 파라미터 업데이트
        optimizer.step()

        # 스케줄러로 학습률 감소
        scheduler.step()

        # 그래디언트 초기화
        model.zero_grad()

    # 평균 로스 계산
    avg_train_loss = total_loss / len(train_dataloader)            

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(format_time(time.time() - t0)))
        
    # ========================================
    #               Validation
    # ========================================

    print("")
    print("Running Validation...")

    #시작 시간 설정
    t0 = time.time()

    # 평가모드로 변경
    model.eval()

    # 변수 초기화
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0

    # 데이터로더에서 배치만큼 반복하여 가져옴
    for batch in validation_dataloader:
        # 배치를 GPU에 넣음
        batch = tuple(t.to(device) for t in batch)
        
        # 배치에서 데이터 추출
        b_input_ids, b_input_mask, b_labels = batch
        
        # 그래디언트 계산 안함
        with torch.no_grad():     
            # Forward 수행
            outputs = model(b_input_ids, 
                            token_type_ids=None, 
                            attention_mask=b_input_mask)
        
        # 로스 구함
        logits = outputs[0]

        # CPU로 데이터 이동
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        
        # 출력 로짓과 라벨을 비교하여 정확도 계산
        tmp_eval_accuracy = flat_accuracy(logits, label_ids)
        eval_accuracy += tmp_eval_accuracy
        nb_eval_steps += 1

    print("  Accuracy: {0:.2f}".format(eval_accuracy/nb_eval_steps))
    print("  Validation took: {:}".format(format_time(time.time() - t0)))

print("")
print("Training complete!")


======== Epoch 1 / 4 ========
Training...

  Average training loss: 0.70
  Training epcoh took: 0:00:09

Running Validation...
  Accuracy: 0.47
  Validation took: 0:00:00

======== Epoch 2 / 4 ========
Training...

  Average training loss: 0.70
  Training epcoh took: 0:00:06

Running Validation...
  Accuracy: 0.47
  Validation took: 0:00:00

======== Epoch 3 / 4 ========
Training...

  Average training loss: 0.70
  Training epcoh took: 0:00:06

Running Validation...
  Accuracy: 0.47
  Validation took: 0:00:00

======== Epoch 4 / 4 ========
Training...

  Average training loss: 0.70
  Training epcoh took: 0:00:06

Running Validation...
  Accuracy: 0.37
  Validation took: 0:00:00

Training complete!


# 테스트셋 평가

In [162]:
#시작 시간 설정
t0 = time.time()

# 평가모드로 변경
model.eval()

# 변수 초기화
eval_loss, eval_accuracy = 0, 0
nb_eval_steps, nb_eval_examples = 0, 0

# 데이터로더에서 배치만큼 반복하여 가져옴
for step, batch in enumerate(test_dataloader):
    # 경과 정보 표시
    if step % 100 == 0 and not step == 0:
        elapsed = format_time(time.time() - t0)
        print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(test_dataloader), elapsed))

    # 배치를 GPU에 넣음
    batch = tuple(t.to(device) for t in batch)
    
    # 배치에서 데이터 추출
    b_input_ids, b_input_mask, b_labels = batch
    
    # 그래디언트 계산 안함
    with torch.no_grad():     
        # Forward 수행
        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask)
    
    # 로스 구함
    logits = outputs[0]

    # CPU로 데이터 이동
    logits = logits.detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()
    
    # 출력 로짓과 라벨을 비교하여 정확도 계산
    tmp_eval_accuracy = flat_accuracy(logits, label_ids)
    eval_accuracy += tmp_eval_accuracy
    nb_eval_steps += 1

print("")
print("Accuracy: {0:.2f}".format(eval_accuracy/nb_eval_steps))
print("Test took: {:}".format(format_time(time.time() - t0)))


Accuracy: 0.50
Test took: 0:00:01


# 새로운 문장 테스트

In [163]:
# 입력 데이터 변환
def convert_input_data(sentences):

    # BERT의 토크나이저로 문장을 토큰으로 분리
    tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]

    # 입력 토큰의 최대 시퀀스 길이
    MAX_LEN = 128

    # 토큰을 숫자 인덱스로 변환
    input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
    
    # 문장을 MAX_LEN 길이에 맞게 자르고, 모자란 부분을 패딩 0으로 채움
    input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

    # 어텐션 마스크 초기화
    attention_masks = []

    # 어텐션 마스크를 패딩이 아니면 1, 패딩이면 0으로 설정
    # 패딩 부분은 BERT 모델에서 어텐션을 수행하지 않아 속도 향상
    for seq in input_ids:
        seq_mask = [float(i>0) for i in seq]
        attention_masks.append(seq_mask)

    # 데이터를 파이토치의 텐서로 변환
    inputs = torch.tensor(input_ids)
    masks = torch.tensor(attention_masks)

    return inputs, masks

In [164]:
# 문장 테스트
def test_sentences(sentences):

    # 평가모드로 변경
    model.eval()

    # 문장을 입력 데이터로 변환
    inputs, masks = convert_input_data(sentences)

    # 데이터를 GPU에 넣음
    b_input_ids = inputs.to(device)
    b_input_mask = masks.to(device)
            
    # 그래디언트 계산 안함
    with torch.no_grad():     
        # Forward 수행
        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask)

    # 로스 구함
    logits = outputs[0]

    # CPU로 데이터 이동
    logits = logits.detach().cpu().numpy()

    return logits

In [165]:
logits = test_sentences(['더 나은 학교생활 하고 싶어'])
print(logits)

if np.argmax(logits) == 1 :
    print("연애 관련 대화")
elif np.argmax(logits) == 0 :
    print("일상 대화")

[[0.17283234 0.16009946]]
일상 대화


In [166]:
logits = test_sentences(['저녁 뭘 먹을지 추천해줘'])

print(logits)
if np.argmax(logits) == 1 :
    print("연애 관련 대화")
elif np.argmax(logits) == 0 :
    print("일상 대화")

[[0.17177275 0.16132557]]
일상 대화


In [168]:
logits = test_sentences(['여자친구한테 선물 뭘로 줄까?'])

print(logits)
if np.argmax(logits) == 1 :
    print("연애 관련 대화")
elif np.argmax(logits) == 0 :
    print("일상 대화")

[[0.17058024 0.15684107]]
일상 대화


# KoBert 모델 만들기

## 1. Colab 환경 설정

In [171]:
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-uavin68_
  Running command git clone --filter=blob:none --quiet 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-uavin68_
  Resolved https://****@github.com/SKTBrain/KoBERT.git to commit 47a69af87928fc24e20f571fe10c3cc9dd9af9a3
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 KB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 344.5/344.5 KB 3.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.7/54.7 MB 18.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 83.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 70.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╸ 881.9/881.9 MB 205.0 MB/s eta 0:00

In [202]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook

#kobert
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

#transformers
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [177]:
#BERT 모델, Vocabulary 불러오기
bertmodel, vocab = get_pytorch_kobert_model()

using cached model. /content/.cache/kobert_v1.zip
using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


## 2. 데이터셋 전처리

## 3.Train data & Test data

In [179]:
print(len(train))
print(len(test))

300
100


## 4.KoBERT 입력 데이터로 만들기

In [190]:
train[:5]

,title,new_article,news
247,허은아 “친윤이고 검사 출신이면 당협 쇼핑하는 현실 부끄럽다”,은 국민 힘 의원 29일 이준석 대표 시절 내정 됐던 서울 동대문 당원 협의 회 당...,0
110,"與 당권주자들, 새해벽두부터 줄줄이 출정식",권성동 1월 6일 출마 선언 계획 안철수 늦어도 설연휴 전 밝힐듯 2023년 새해 ...,1
16,"與조강특위, 조직위원장 42명 선임…‘이재명 저격수’ 장영하 포함",국민 힘 조직 강화 특별 위원회 조강 특위 는 29일 총 42 명의 국회의원 선거구...,1
66,尹 “전쟁 준비” 발언에…이재명 “안보무능 정권의 철부지 행동”,민주당 윤석열 대통령 전쟁 준비 발언 두고 안보 불안 부추기고 며 공세 나섰다 이재...,1
153,野 “1월 임시국회 소집 불가피”…與 “이재명 방탄국회 열겠다는 것”,이재명 민주당 대표 박찬 대 최고 위원 지난 23일 오후 서울 여의도 국회 열린 본...,1


In [192]:
# 토큰화 & 패딩
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)
        self.sentences = [transform([i[sent_idx]]) for i in dataset]

        labels = test['news'].values
        self.labels = labels.astype(np.int)

        # self.labels = [np.int32(i[label_idx]) for i in dataset]
        

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [193]:
# 하이퍼파라미터 조정 Batch size는 64, epochs는 5, learning rate는 5e-5
# Setting parameters
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [194]:
#토큰화
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

data_train = BERTDataset(train, 1, 2, tok, max_len, True, False)
data_test = BERTDataset(test, 1, 2, tok, max_len, True, False)

using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


<ipython-input-192-e65f04dac005>:10: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  self.labels = labels.astype(np.int)


In [196]:
# 첫 번째는 패딩된 시퀀스, 두 번째는 길이와 타입에 대한 내용, 세 번째는 어텐션 마스크 시퀀스
data_train[0]

(array([  2, 517, 405,   3,   1,   1,   1,   1,   1,   1,   1,   1,   1,
          1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,
          1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,
          1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,
          1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1],
       dtype=int32),
 array(4, dtype=int32),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       dtype=int32),
 0)

In [197]:
# torch 형식의 dataset을 만들어준다.
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

/usr/local/lib/python3.8/dist-packages/torch/utils/data/dataloader.py:554: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  assert self._num_workers == 0


## 5. KoBERT 학습 모델 만들기

In [200]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=2,   ##클래스 수 조정##
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [203]:
#BERT 모델 불러오기
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

#optimizer와 schedule 설정
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

#정확도 측정을 위한 함수 정의
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc
    
train_dataloader

## 6. KoBERT 모델 학습시키기

In [204]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

<ipython-input-204-480b6a139979>:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):


  0%|          | 0/2 [00:00<?, ?it/s]

IndexError: ignored